## Business Challege

Create a product that build a brouchure for a company to be used for prospective clients, investers and potential recruits.

We will be provided a company name and their primary webiste.

See the end of this notebook for examples of real time business applications.

In [2]:
!pip install openai

In [3]:
#imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("API key looks good so far")
else: 
    print("There might be problem with your API key?")
    

API key looks good so far


In [5]:
# A class to represent a WebPage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    test: str
    links: List[str]
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "imput"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    

In [6]:
website = Website("https://www.bvuniversity.edu.in/")
print(website.get_contents())

Webpage Title:

	Best University in Mumbai, Pune, Maharashtra | Bharati Vidyapeeth

Webpage Contents:
Examinations
Apply Now
Admission Enquiry
About
Programmes
All Programmes
UG Programmes
PG Programmes
PG Diploma Programmes
Ph.D Programmes
Fellowship Programmes
Certificate Programmes
Diploma Programmes
Short Term Programmes
Distance Education
Online Education
Colleges
Pune Dhankawadi Campus
School of Optometry, Pune
School of Physiotherapy, Pune
College of Architecture
College of Ayurved
College of Engineering
College of Nursing
College of Physical Education
Dental College & Hospital
Homoeopathic Medical College
Institute of Environment Education & Research
Institute of Hotel Management & Catering Technology
Medical College
Rajiv Gandhi Institute of Information Technology & Bio-Technology
School of Visual Arts, Pune
School of Audiology and Speech Language Pathology, Pune
Pune Erandwane Campus
Institute of Management & Entrepreneurship Development
New Law College
Poona College of Pharm

In [7]:
website.links

['https://www.bvuniversity.edu.in/Uploads/moduleimg/10409imguf_NAAC_certificate_4th_cycle_181224.pdf',
 '/',
 'https://bvuict.in/exam/index1.html',
 'https://bvducet.bharatividyapeeth.edu/',
 'https://www.bvuniversity.edu.in/admission-enquiry',
 '/about/bharati-vidyapeeth',
 '/programmes',
 '/programmes#locationwise',
 '/ug-programmes#locationwise',
 '/pg-programmes#locationwise',
 '/pg-diploma-programmes#locationwise',
 '/research-programmes#locationwise',
 '/fellowship-programmes#locationwise',
 '/certification-programmes#locationwise',
 '/diploma-programmes#locationwise',
 '/short-term-courses#locationwise',
 '/distance-education#locationwise',
 '/online-education#locationwise',
 'javascript:void(0);',
 '/optometry/',
 '/soptpune/',
 '/coarcpune/',
 'https://coayurved.bharatividyapeeth.edu/',
 '/coepune/',
 '/conpune/',
 '/copepune/',
 '/dchpune/',
 '/hmc/',
 '/ieer/',
 '/ihmct/',
 'https://mcpune.bharatividyapeeth.edu/',
 '/rgitbt/',
 '/visualarts/',
 '/aslp/',
 '/imed/',
 '/nlcpun

## 
Use a call gpt-4o-mini to read the links on webpage, and respond in structured JSON

In [9]:
#System Prompt

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [11]:
#User_prompt

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(website))

Here is the list of links on the website of https://www.bvuniversity.edu.in/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.bvuniversity.edu.in/Uploads/moduleimg/10409imguf_NAAC_certificate_4th_cycle_181224.pdf
/
https://bvuict.in/exam/index1.html
https://bvducet.bharatividyapeeth.edu/
https://www.bvuniversity.edu.in/admission-enquiry
/about/bharati-vidyapeeth
/programmes
/programmes#locationwise
/ug-programmes#locationwise
/pg-programmes#locationwise
/pg-diploma-programmes#locationwise
/research-programmes#locationwise
/fellowship-programmes#locationwise
/certification-programmes#locationwise
/diploma-programmes#locationwise
/short-term-courses#locationwise
/distance-education#locationwise
/online-education#locationwise
javascript:void(0);
/optometry/
/soptpune/
/coarcpune/
https://coayurved.bh

In [13]:
openai = OpenAI()

In [14]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [15]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main-content',
 '#footer',
 '/',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/core-views-on-ai-safety',
 'https://www.anthropic.com/rsp-updates',
 '/learn',
 '/research/tracing-thoughts-language-model',
 '/economic-index',
 '/news/claude-3-7-sonnet',
 '/research/visible-extended-thinking',
 '/research/alignment-faking',
 '/news/model-context-protocol',
 '/jobs',
 '/',
 '/claude',
 'https://anthropic.com/claude-code',
 '/team',
 '/enterprise',
 '/education',
 'https://claude.ai/download',
 '/pricing',
 'http://claude.ai/login',
 '/api',
 'https://docs.anthropic.com/',
 '/amazon-bedrock',
 '/google-cloud-vertex-ai',
 '/pricing#api',
 'https://console.anthropic.com/',
 '/research',
 '/economic-index',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-3-family',
 '/transparency',
 '/responsible-scaling-policy',
 'https://trust.anthropic.com',
 '/solutions/agents',
 '/s

In [16]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

## Make a brochure!

Assemble all the details into another  prompt to GPT4-o

In [34]:
def get_all_details(url):
    result = "Landing page: \n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found likns:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [36]:
print(get_all_details("https://www.anthropic.com"))

Found likns: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}
Landing page: 
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
API
Solutions
Research
Commitments
Learn
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
At Anthropic, we build AI to serve humanity’s long-term well-being.
While no one can foresee every outcome AI will have on society, we do know that designing powerful technologies requires both bold

In [77]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [42]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this infomation to build a short brochure of the company in markdown. \n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]   #truncate if more than 20,000 characters 
    return user_prompt

In [52]:
get_brochure_user_prompt("Anthropic", "https://www.anthropic.com")

Found likns: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this infomation to build a short brochure of the company in markdown. \nLanding page: \nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nAPI\nSolutions\nResearch\nCommitments\nLearn\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.7 Sonnet\nClaude 3.7 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAt Anthropic, we build AI to serve humanity’s long-term well-being.\nWhile no one can foresee every outcome AI will have on society, we do know that designing powerful technologies requires both bold steps forward and intentional pauses to consider the effects. That’s why we focus on building tools with human benefit at their foundation, 

In [79]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [87]:
create_brochure("Anthropic", "https://www.anthropic.com")

Found likns: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}]}


# Welcome to Anthropic: Your Friendly AI Safety Squad! 

**Where love for AI meets safety and a dash of humor!** 😄

---

## Who Are We?

At Anthropic, we're not just creating AI; we're going on an epic quest to build **reliable, interpretable, and steerable AI systems**! Whether it's the next creative sonnet from our star model Claude 3.7 or coding the future of AI, we're on a mission to ensure that technology works for the benefit of mankind. Think of us as the protective guardians of the AI universe—or at least the cool ones in the corner making sure robots stay friendly!

---

## Our Customers: The Coolest Users Around! 🌍

- **Businesses** looking to harness the power of AI without losing their sanity.
- **Nonprofits** needing smart tools to make a real impact—while still having time for coffee breaks.
- **Civilians** (that’s you!) curious about how AI can make life a little easier and a lot more fun!

---

## Join the Fun: Careers Worth Buzzing About! 🐜

Want to feel like a superhero while working? At Anthropic, we hold firm to our **employee-centric culture**! Here’s what you get when you join our team:

- **Health & Wellness Benefits**: Because we want you thriving! With health, dental, and vision insurance, flexible paid time off, and mental health support—you're covered!
- **Individuality Appreciated**: No PhDs? No problem! Talent comes in many forms, and we love a good variety.
- **Generous Stipends**: Feeling fancy? We have stipends for wellness, food, and even your home office setup!
- **Plant Lovers**: Yep, we don't just appreciate our employees, we love our leafy friends too! 🌱

Remember: if you’ve ever written a blog post that blew people away or crafted a tool that made life easier, bring it to a stellar position at Anthropic!

---

## Our Values: More Fun than Just Beeping Machines 🔧

**1. Act for the Global Good**: We aren’t just here to have a good time; we aim to maximize long-term impact. You're invited to play a pivotal role!

**2. Hold Light and Shade**: We balance risks with rewards like experts; all in a day’s work.

**3. Be Good to Our Users**: Kindness is key! Whether you're an Ant (that’s our fun name for team members) or a customer, you matter.

**4. Ignite a Race to the Top on Safety**: We want others in the AI space to push for higher safety standards—like a friendly competition but without the awkward trophies.

**5. Simple Solutions, Big Impact**: We’re in favor of efficient solutions. After all, why build a spaceship when a bicycle will suffice?

---

## Talking Tech: Our Magic Maker, Claude! 🧙‍♂️

Meet **Claude**, your AI magician! Whether you're crafting the next big text or conjuring up sophisticated applications, Claude’s here to serve humanity's long-term well-being—it's in our *AI DNA*!

---

## Connect with Us

Curious to learn more or join our quest? 🧑‍🚀

- Dive into our [Careers Page](#) to see job openings.
- Follow us on social media for updates, memes, and more fun (because who doesn’t love a good AI pun?).
- Send us a non-media inquiry at [support@anthropic.com](mailto:support@anthropic.com).

---

## Anthropic: ***Building a Safer Future—One AI at a Time!***

*Because who said saving the world couldn't be a little entertaining?* 🌟

--- 

**Disclaimer**: Anthropic accepts no responsibility for any robot uprising caused while utilizing our AI systems. Just kidding! We're all about the good vibes!

# Minor improvement

In [83]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream = True
    )
    # for chunk in stream:
    #     print(chunk.choices[0].delta.content)

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("'''", "").replace("markdown", "")
        update_display(Markdown(response), display_id = display_handle.display_id)

In [84]:
stream_brochure("Anthropic", "https://www.anthropic.com")

Found likns: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}]}


# Welcome to Anthropic: Where AI Meets Humanity!

---

## Meet Claude
Not just a name, but a promise! Claude 3.7 Sonnet is our most intelligent AI model, here to do more than just carry your groceries. Whether you want to chat or create AI-powered experiences, Claude's got your back (and your shopping list).

---

## What’s Our Mission?
At Anthropic, we’re serious about building AI that serves humanity’s long-term well-being. Our AI doesn't just calculate; it contemplates life, the universe, and what’s for dinner—all while keeping safety at the forefront. We believe AI can be like a warm hug — if a hug could analyze data at lightning speed!

---

## Company Culture: The Good, The Bold, and The Thoughtful
### Our Values:
1. **Act for the Global Good**: We make AI that takes humanity on a joyride through the technological revolution.
2. **Hold Light and Shade**: We know AI can be spooky or spiffy. We tackle the dark and celebrate the dazzling!
3. **Be Good to Our Users**: Everyone from policy-makers to ants (yes, those little ones) matters in our world.
4. **Ignite a Race to the Top on Safety**: It's like a track meet, but everyone wins when safety is the goal!
5. **Do the Simple Thing That Works**: Why invent a jetpack when a bicycle meets the needs?
6. **Be Helpful, Honest, and Harmless**: Think of us as the friendly neighbor—always willing to lend a cup of sugar, or in our case, some insightful AI analysis.
7. **Put the Mission First**: Less multitasking, more impactful action! We’re united in our quest to make the world a safer, smarter place.

---

## Our Team: The Avengers of AI Safety
Our diverse crew isn’t just smart; they're *super* smart! With backgrounds in physics, public policy, and even space exploration, our team works together like peanut butter and jelly. But instead of sandwiches, we’re making reliable, interpretable AI systems.

---

## Why Join Us?
At Anthropic, we offer more than just a job. We're about creating environments where you can thrive while feeling secure. Here’s what you get:
- **Health & Wellness**: Health insurance that covers your puppy (just kidding, kind of).
- **Parental Leave**: 22 weeks of paid parental leave — because babies can’t raise themselves, right?
- **Generous Stipends**: Want to go to yoga or buy a new desk? We got you covered with flexible stipends!
- **Daily Meals & Snacks**: Because who doesn't love free food? (seriously, we have snacks *every* day).

---

## Careers: Are You Our Next Hero?
Whether you’re a codex-crunching coder or a policy pondering pro, we want YOU to help us build a better tomorrow. Multiple roles are open; we’re all about that diverse talent!

*Not required: PhDs or machine-learning experience, though we love sharing our lunch with space nerds and policy pros too!*

---

## Join the Adventure!
So, ready to be part of the quest to develop safe and user-friendly AI? Whether you're looking to partner, invest, or just chat over a virtual coffee, we’d love to hear from you!

---

> **Anthropic**: Where AI isn't just a tool; it's a friendly co-pilot in our journey towards humanity's future! 🚀

In [75]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found likns: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: The AI Community Building the Future

## About Us
Hugging Face is the leading collaborative platform in the AI and machine learning community. Our mission is to democratize machine learning and empower individuals and organizations by providing a space to create, discover, and collaborate on models, datasets, and applications. With over 1 million models and 250,000 datasets, we are the home for cutting-edge machine learning research and deployment.

### Core Values:
- **Community-Centric**: We believe in the power of community contributions and knowledge sharing.
- **Open Source**: Our commitment to open-source principles is at the heart of everything we do.
- **Innovation**: We continuously strive to improve our tools and technology to drive advancements in AI.

## Company Culture
At Hugging Face, we foster a culture of inclusivity, collaboration, and innovation. Our diverse team of professionals from various backgrounds works together to push the boundaries of what's possible in AI. We value creativity and encourage our team members to contribute ideas and solutions freely. As part of our culture, we prioritize **work-life balance** and support personal and professional growth through continuous learning opportunities.

## Customers
We serve a wide range of clients, including over 50,000 organizations worldwide, from **startups** to **Fortune 500 companies** such as Google, Amazon, Microsoft, and Intel. Our platform supports various industries, allowing businesses to leverage the power of AI for their unique needs.

### Notable Customers Include:
- **Meta**: Utilizing AI to enhance user experience.
- **Tesla**: Implementing AI for automation and intelligence.
- **Grammarly**: Enhancing writing with cutting-edge language models.

## Career Opportunities
We are always on the lookout for innovative minds to join our team. At Hugging Face, we offer a collaborative and dynamic work environment where you can make a real impact. 

### Current Openings:
- Data Scientists
- Backend & Frontend Engineers
- Machine Learning Researchers
- Community Managers

Explore our job openings and apply to be part of the next wave of AI innovation. 

### Why Work with Us?
- **Flexible Work Environment**: Remote-first culture that encourages a healthy work-life balance.
- **Growth and Learning Opportunities**: Access to conferences, workshops, and further education.
- **Unique Projects**: Contribute to transformative AI projects that influence millions.

## Join Us
Whether you are looking to enhance your machine learning capabilities, collaborate on groundbreaking projects, or join a diverse team of AI enthusiasts, Hugging Face offers the tools and community for success. 

### Get Started Today:
- **Explore our Models**: Find what fits your needs [here](https://huggingface.co/models).
- **Join the Community**: Sign up and start collaborating with us [here](https://huggingface.co/join).

Together, we can build the future of AI!